In [1]:
import numpy as np

import requests
from bs4 import BeautifulSoup
import pickle

from pymongo import MongoClient
import psycopg2
import time

from datetime import datetime
import json
import time
import re

In [2]:
def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

In [3]:
def get_job_info(soup):

    try:
        job_title = soup.find(name='h3', attrs={
            'class':'icl-u-xs-mb--xs icl-u-xs-mt--none \
            jobsearch-JobInfoHeader-title'}).text
    except:
        job_title = ''
        
    try:
        company_name = soup.find(name='div', attrs={
            'class':'icl-u-lg-mr--sm icl-u-xs-mr--xs'}).text
    except:
        company_name = ''
        
    try:
        salary = soup.find(name='span', attrs={'class':'icl-u-xs-mr--xs'}).text
    except:
        salary = ''
    
    try:
        job_desc_full = soup.find(name='div',attrs={
            'class':'jobsearch-JobComponent-description icl-u-xs-mt--md'})
        
        try:
            job_desc_full = soup.find(name='div', attrs={
                'class':'jobsearch-JobComponent-description icl-u-xs-mt--md'})
            listed_items = []
            for list_item in job_desc_full.find_all(name='li'):
                listed_items.append(list_item.text)
            listed_items = '. '.join(listed_items)
            listed_items = re.sub('\n| +|\.\.', ' ', listed_items)
        except:
            listed_items = ''
         
        try:
            paragraph_items = []
            for paragraph_item in job_desc_full.find_all(name='p'):
                paragraph_items.append(paragraph_item.text)

            # Remove partially captured descriptions
            paragraph_items = [paragraph_item for paragraph_item in
                               paragraph_items if len(paragraph_item) > 50]
            # Puts all paragraph items into a single string
            paragraph_items = ''.join(paragraph_items)
            paragraph_items = re.sub('\n', ' ', paragraph_items)

            if len(paragraph_items) < 1:
                try:
                    job_desc_full = soup.find(name='div',attrs={
                        'class':
                        'jobsearch-JobComponent-description icl-u-xs-mt--md'})
                    paragraph_items = []
                    for paragraph_item in job_desc_full.find_all(name='div'):
                        paragraph_items.append(paragraph_item.text)

                    paragraph_items = [paragraph_item for paragraph_item in
                                       paragraph_items if len(paragraph_item)
                                       > 50]
                    paragraph_items = ''.join(paragraph_items)
                    paragraph_items = re.sub('\n', ' ', paragraph_items)
                except: 
                    paragraph_items = ''
   
        except:
            paragraph_items = ''
    
    except:
        listed_items, paragraph_items = ['','']
            
    return job_title, company_name, salary, listed_items, paragraph_items

### Connecting to MongoDB

In [4]:
client = MongoClient()
db = client.jobs_data
db.list_collection_names()

['Chicago_Jobs',
 'Los_Angeles_Jobs',
 'SF_Bay_Area_Jobs',
 'Seattle_Jobs',
 'New_York_Jobs']

In [5]:
pickles = [
    'individual_listings__chicago',
    'individual_listings__LosAngeles',
    'individual_listings__NewYork',
    'individual_listings__SanFranciscoBayArea',
    'individual_listings__Seattle'
]
collections = [
    'Chicago_Jobs',
    'Los_Angeles_Jobs',
    'New_York_Jobs',
    'SF_Bay_Area_Jobs',
    'Seattle_Jobs'
]

In [ ]:
i = 0

date = datetime.strftime(datetime.today(), "%Y-%m-%d")

with open('conn.json') as fp:
    conn_kwargs = json.load(fp)

conn = psycopg2.connect(**conn_kwargs)
cur = conn.cursor()

cur.execute("""
SELECT DISTINCT(link), query, city
FROM jobs_pages
WHERE city ILIKE 'San+Francisco+Bay+Area%2C+CA';""")
jobs_urls = cur.fetchall()

for url, query, city in jobs_urls:

    new_item = {}

    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    job_title, company_name, salary, listed_items, paragraph_items \
    = get_job_info(soup)
    time.sleep(np.random.poisson(100)/50)

    new_item['date'] = date
    new_item['url'] = url
    new_item['query'] = query
    new_item['city'] = city
    new_item['job_title'] = job_title 
    new_item['company_name'] = company_name
    new_item['salary'] = salary
    new_item['listed_items'] = listed_items
    new_item['paragraph_items'] = paragraph_items

    time.sleep(np.random.poisson(100)/50)

    db.SF_Bay_Area_Jobs.insert_one(new_item)

    if i % 100 == 0:
        print(i)

    i += 1

In [9]:
cur.execute("""
SELECT DISTINCT(link), query, city
FROM jobs_pages
WHERE city ILIKE 'San+Francisco+Bay+Area%2C+CA';
""")
jobs_urls = cur.fetchall()

In [10]:
len(jobs_urls)

11068